In [9]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import AutoTokenizer
import random
import copy
import time

In [10]:
from transformers import XLMTokenizer, XLMWithLMHeadModel

tokenizer = XLMTokenizer.from_pretrained("xlm-mlm-enfr-1024")
pre_model = XLMWithLMHeadModel.from_pretrained("xlm-mlm-enfr-1024")

Some weights of XLMWithLMHeadModel were not initialized from the model checkpoint at xlm-mlm-enfr-1024 and are newly initialized: ['transformer.position_ids']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# Load data here

In [11]:
data_en = open("data/hansards.e", encoding='utf-8').read().split('\n')
data_fr = open("data/hansards.f", encoding='utf-8').read().split('\n')


raw_data = {'en': [line for line in data_en], 'fr': [line for line in data_fr]}

df = pd.DataFrame(raw_data, columns = ['en', 'fr'])
df_small = df[['en', 'fr']][:100]

## All transformer classes

In [12]:
class Embedder(nn.Module):
    def __init__(self, vocab_size, d_model):
        super().__init__()
        
        self.embed = nn.Embedding(vocab_size, d_model)
        
    def forward(self, x):
        return self.embed(x)


class PositionalEncoder(nn.Module):
    def __init__(self, d_model, max_seq_len=1024):
        super().__init__()
        
        self.d_model = d_model
        
        # Allocate memory to 
        pe = torch.zeros((max_seq_len, d_model))
        
        ### From attention is all you need ###
        for pos in range(max_seq_len):
            for i in range(0,d_model,2):
                pe[pos, i] = np.sin(pos/10000**(2*i/self.d_model))
                pe[pos, i+1] = np.cos(pos/10000**(2*i/self.d_model))
        # Fixed positional encoding
        pe.requires_grad = False
        pe = pe.unsqueeze(0) # Make pe into [batch size x seq_len x d_model]
        self.register_buffer('pe',pe)
        
    def forward(self,x):

        # Make embeddings larger
        x = x*np.sqrt(self.d_model)
        # Get sequence length
        seq_len = x.size(1)
        x = x + torch.autograd.Variable(self.pe[:,:seq_len], 
        requires_grad=False)
        return x


def Attention(Q, K, V, d_k, mask=None, dropout=None):

    vals = (Q @ K.transpose(-2,-1))/np.sqrt(d_k)
    
    # Mask the scores if mask is specified. Model cannot see into future if masked.
    if mask is not None:
        mask = mask.unsqueeze(1)
        vals = vals.masked_fill(mask, 1e-5)
    # vals = vals if mask is None else vals.masked_fill_(mask, 1e-4)
    
    softmax = nn.Softmax(dim=-1)
    vals = softmax(vals)
    
    # apply dropout if specified
    vals = vals if dropout is None else dropout(vals)
    
    out =  vals @ V
    return out


class MultiHeadAttention(nn.Module):
    def __init__(self, n_heads, d_model, d_k, dropout=.1, relative = False):
        super().__init__()

        self.n_heads = n_heads
        self.d_model = d_model
        # self.seq_len = seq_len
        self.d_k = d_k
        
        self.linear = nn.Linear(d_model, d_model)
        self.dropout = nn.Dropout(dropout)
        self.out = nn.Linear(d_model, d_model)
    
    # d_model = 512
    # n_heads = 8
    # seq_len = 20
    
    # [20,512] --> [20, 8, 64]
    ## If batch size is used, say of 128:
    ## out = [128, 20, 8, 64]
    
    # Input = Matrix of dim [bs x seq_len x d_model]
    def split_heads(self, t):
        return t.reshape(t.size(0), -1, self.n_heads, int(self.d_k))
    # Output = Matrix of dim [bs x seq_len x n_heads x d_k]
    
    def forward(self, Q, K, V, mask = None):
        
        Q = self.linear(Q)
        K = self.linear(K)
        V = self.linear(V)
        
        Q, K, V = [self.split_heads(t) for t in (Q, K, V)] 
        Q, K, V = [t.transpose(1,2) for t in (Q, K, V)] # reshape to [bs x n_heads x seq_len x d_k]
        
        # Compute Attention
        vals = Attention(Q, K, V, self.d_k, mask, self.dropout)
        
        # Reshape to [bs x seq_len x d_model]
        vals = vals.transpose(1,2).contiguous().view(vals.size(0), -1, self.d_model)
       
        out = self.out(vals) # linear
        return out
    
class FeedForwardNetwork(nn.Module):
    def __init__(self, d_model, d_ff, dropout=.1):
        super().__init__()
        
        self.linear1 = nn.Linear(d_model, d_ff)
        self.linear2 = nn.Linear(d_ff, d_model)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):
        
        return self.linear2(self.dropout(F.relu(self.linear1(x))))
    

        
class EncoderLayer(nn.Module):
    def __init__(self, n_heads, d_model, d_ff, d_k, dropout=.1):
        
        super().__init__()
        self.MHA = MultiHeadAttention(n_heads, d_model, d_k, dropout)
        self.FFN = FeedForwardNetwork(d_model, d_ff, dropout)
        
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.dropout1 = nn.Dropout(dropout)
        self.dropout2 = nn.Dropout(dropout)
    
    def forward(self, x, mask=None):
        # See "Attention is all you need" to follow code structure
        x2 = self.dropout1(self.MHA(x, x, x, mask))
        x = self.norm1(x) + self.norm1(x2)
        
        x2 = self.dropout2(self.FFN(x))
        x = x + self.norm2(x2)
    
        return x
    

class DecoderLayer(nn.Module):
    def __init__(self, n_heads, d_model, d_ff, d_k, dropout=.1):
        super().__init__()
        self.d_model = d_model
        self.n_heads = n_heads
        
        self.MHA = MultiHeadAttention(n_heads, d_model, d_k, dropout)
        self.FFN = FeedForwardNetwork(d_model, d_ff, dropout)
        
        # Batch Normalization
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.norm3 = nn.LayerNorm(d_model)
        
        # Dropout
        self.dropout1 = nn.Dropout(dropout)
        self.dropout2 = nn.Dropout(dropout)
        self.dropout3 = nn.Dropout(dropout)
        
        # self.linear = nn.Linear()
        
    def forward(self, x, e_out, source_mask, target_mask):
        
        # See "Attention is all you need" to follow code structure
        ## part 1
        x2 = self.norm1(x) # Norm
        x = self.dropout1(self.MHA.forward(x2, x2, x2, target_mask)) # Masked MHA, target
        x = x2 + self.norm1(x) # Add & Norm
        
        ## part 2
        x3 = self.dropout2(self.MHA.forward(x, e_out, e_out, source_mask)) # MHA on encoder output
        x2 = self.dropout2(self.MHA.forward(x, x, x)) #MHA continued in decoder
        x = self.norm2(x3) + self.norm2(x2) + self.norm2(x) # Add & Norm
        
        ## part 3
        x2 = self.dropout3(self.FFN.forward(x)) ## Feed forward
        
        x = x + self.norm3(x2) # add
        # x = self.norm3(x) # norm (!!!CHECK IF THIS IS EQUIVALENT!!!)
        return x

def cloneLayers(module, n_layers):
    return nn.ModuleList([copy.deepcopy(module) for i in range(n_layers)])

class Encoder(nn.Module):
    def __init__(self, vocab_size, d_model, d_ff, d_k, n_layers, n_heads, dropout=.1):
        super().__init__()
        self.n_layers = n_layers
        self.embedder = Embedder(vocab_size, d_model)
        self.pe = PositionalEncoder(d_model)
        self.e_layers = cloneLayers(EncoderLayer(n_heads, d_model, d_ff, d_k), n_layers)
        self.norm = nn.LayerNorm(d_model)
        
        
    def forward(self, source, mask=None):
        x = self.embedder.forward(source)
        x = self.pe.forward(x)
        for i in range(self.n_layers):
            x = self.e_layers[i](x, mask)
        
        return self.norm(x)

class Decoder(nn.Module):
    def __init__(self, vocab_size, d_model, d_ff, d_k, n_layers, n_heads, dropout=.1):
        super().__init__()
        self.n_layers = n_layers
        self.embedder = Embedder(vocab_size, d_model)
        self.pe = PositionalEncoder(d_model)
        self.d_layers = cloneLayers(DecoderLayer(n_heads, d_model, d_ff, d_k), n_layers)
        self.norm = nn.LayerNorm(d_model)
    
    def forward(self, trg, e_out, source_mask, target_mask):
        x = self.embedder.forward(trg)
        x = self.pe.forward(x)
        
        for i in range(self.n_layers):
            x = self.d_layers[i](x, e_out, source_mask, target_mask)
        
        return self.norm(x)
        
class AlignmentLayer(nn.Module):
    def __init__(self, source_vocab_size, target_vocab_size, d_model, d_ff, d_k, n_layers, n_heads):
        super().__init__()
        self.MHA = MultiHeadAttention(n_heads, d_model, d_k)
        self.e = Encoder(source_vocab_size, d_model,d_ff, d_k, n_layers, n_heads)
        self.d = Decoder(target_vocab_size, d_model,d_ff, d_k, n_layers, n_heads)
        self.linear_f = nn.Linear(d_model, target_vocab_size) 
        
    def forward(self, source, target, source_mask, target_mask):
        e_out = self.e.forward(source, source_mask)
        d_out = self.d.forward(target, e_out, source_mask, target_mask)
        
        out = self.MHA.forward(d_out, e_out, e_out)
        
        
        
class Transformer(nn.Module):
    def __init__(self, source_vocab_size, target_vocab_size, d_model,d_ff, d_k, n_layers, n_heads):
        super().__init__()
        self.e = Encoder(source_vocab_size, d_model,d_ff, d_k, n_layers, n_heads)
        self.d = Decoder(target_vocab_size, d_model,d_ff, d_k, n_layers, n_heads)
        self.linear_f = nn.Linear(d_model, target_vocab_size)
        
    def forward(self, source, target, source_mask, target_mask):
        e_out = self.e.forward(source, source_mask)
        d_out = self.d.forward(target, e_out, source_mask, target_mask)
        
        out = self.linear_f(d_out)
        return out
        


In [13]:
def get_target_mask(size, target):
    mask = (torch.triu(torch.ones(size, size)) == 1).transpose(0, 1)
    mask = mask.float().masked_fill(mask == 0, float(0)).masked_fill(mask == 1, float(1))
    
    # Find out where the target pad starts
    trg_pad = (target==0).nonzero()
    
    # Check if there is no padding in sentence
    if len(trg_pad) == 0:
        stop_idx = size
        
    else:
        stop_idx = trg_pad[0][1].item()
        mask[stop_idx:, :] = -69
        
    return mask.unsqueeze(0) > 0, stop_idx

In [14]:
def get_source_mask(size, source):
    src_pad = (source==0).nonzero()
    
    if len(src_pad) == 0:
        stop_idx = size
        
    else:
        stop_idx = src_pad[0][1].item()
    
    mask = source.clone()
    # Mask all padding with -inf
    mask[:, stop_idx:] = 0
    # Convert everything before stop_idx to zero
    mask[:,:stop_idx] = 1
    
    mask = mask.unsqueeze(0) > 0
    
    return mask

In [15]:
# Tokenizer
model_checkpoint = 't5-small'
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, model_max_length=512)


# Function for mapping data from strings to tokens
# s_key = source key, t_key = target_key, max_length = max sentence length
def preprocess_data(df, s_key, t_key, max_length):
    s = [sentence for sentence in df[s_key]]
    t = [sentence for sentence in df[t_key]]
    
    model_input = tokenizer(s, max_length=max_length, truncation=True, padding=True, return_tensors='pt') 
    
    with tokenizer.as_target_tokenizer():
        target_tokens = tokenizer(t, truncation=True, padding=True, max_length=max_length, return_tensors='pt') 
        
    model_input['target'] = target_tokens['input_ids']
    
    return model_input

# Preprocess data here

In [8]:
data = preprocess_data(df, 'en', 'fr', 80)

print("REMEMBER TO UNCOMMENT THE BELOW CODE!!")
#src_vocab_size = len(np.unique([word for sentence in data['input_ids'] for word in sentence]))
#trg_vocab_size = len(np.unique([word for sentence in data['target'] for word in sentence]))
#vocab_size = src_vocab_size + trg_vocab_size


REMEMBER TO UNCOMMENT THE BELOW CODE!!


In [9]:
data_small = preprocess_data(df_small, 'en', 'fr', 50)

In [16]:
def train_model(model, data, epochs, verbose=False):
    print("Training transformer")
    model.train()
    start = time.time()
    total_loss = 0
    loss_list = []
    
    source_all = data['input_ids']
    target_all = data['target']
    
    # loop over epochs
    for epoch in range(epochs):
        print("epoch", epoch+1)
        # loop over all sentences
        for i in range(len(source_all)):
            if verbose and i % print_every == 0:
                print("sentece", i)
            
            # unsqueeze to avoid dim mismatch between embedder and pe
            src = source_all[i].unsqueeze(0) 
            trg = target_all[i].unsqueeze(0)
            
            # target input, remove last word
            trg_input = trg[:, :-1]
            
            # get targets
            y = trg[:, 1:].contiguous().view(-1)
            
            src_mask = get_source_mask(src.size(1), src)
            trg_mask, trg_stop_idx = get_target_mask(trg_input.size(1), trg_input)
            
            preds = model.forward(src, trg_input, src_mask, trg_mask)
            optim.zero_grad()    
            loss = F.cross_entropy(preds.view(-1, preds.size(-1)), y)
            loss.backward()
            optim.step()
            total_loss += loss.item()
            loss_list.append(loss.item())
            
            if verbose and i % print_every == 0:
                print("time:",np.round(time.time()-start, 2), "\n loss:", np.round(loss.item(),2), "\n total loss:", np.round(total_loss,2)) 
    

In [40]:
### Define arguments ### (same as in "Attention is all you need")
d_model = 512 # Dimension of embeddings
d_k = 64 # dimension of keys (d_model / n_heads)
d_ff = 1024
vocab_size = len(df) # Number of (unique) words in dataset
n_heads = 8 # Number of heads for MHA
n_layers = 6 # Number of model layers
epochs = 1 
model_checkpoint = 't5-small'


print("!!!REMEMBER TO CHANGE EPOCHS!!!")


!!!REMEMBER TO CHANGE EPOCHS!!!


In [41]:
model = Transformer(25000, 25000, d_model, d_ff, d_k, n_layers, n_heads)

for p in model.parameters():
    if p.dim() > 1:
        nn.init.xavier_uniform_(p)

lr = 0.01 # 0.0001 default
optim = torch.optim.Adam(model.parameters(), lr=lr, betas=(0.9, 0.98), eps=1e-9)

In [45]:
m_keys = list(model.state_dict().keys())

cnt = 0
for key in m_keys:
    if "MHA" in key:
        cnt+=1

cnt

48

In [51]:
p_keys = list(pre_model.state_dict().keys())

cnt = 0
cnt2=0
for key in p_keys:
    if "attention" in key:
        cnt+=1
    if "encoder_attn" in key:
        cnt2+=1

print(cnt)
cnt2

48


0

In [44]:

for key, value in model.state_dict().items():
    print(key, value.size())

e.embedder.embed.weight torch.Size([25000, 512])
e.pe.pe torch.Size([1, 1024, 512])
e.e_layers.0.MHA.linear.weight torch.Size([512, 512])
e.e_layers.0.MHA.linear.bias torch.Size([512])
e.e_layers.0.MHA.out.weight torch.Size([512, 512])
e.e_layers.0.MHA.out.bias torch.Size([512])
e.e_layers.0.FFN.linear1.weight torch.Size([1024, 512])
e.e_layers.0.FFN.linear1.bias torch.Size([1024])
e.e_layers.0.FFN.linear2.weight torch.Size([512, 1024])
e.e_layers.0.FFN.linear2.bias torch.Size([512])
e.e_layers.0.norm1.weight torch.Size([512])
e.e_layers.0.norm1.bias torch.Size([512])
e.e_layers.0.norm2.weight torch.Size([512])
e.e_layers.0.norm2.bias torch.Size([512])
e.e_layers.1.MHA.linear.weight torch.Size([512, 512])
e.e_layers.1.MHA.linear.bias torch.Size([512])
e.e_layers.1.MHA.out.weight torch.Size([512, 512])
e.e_layers.1.MHA.out.bias torch.Size([512])
e.e_layers.1.FFN.linear1.weight torch.Size([1024, 512])
e.e_layers.1.FFN.linear1.bias torch.Size([1024])
e.e_layers.1.FFN.linear2.weight torch.

In [38]:
p_vals = list(pre_model.state_dict().values())
for key, value in pre_model.state_dict().items():
    print(key, value.size())

transformer.position_ids torch.Size([1, 512])
transformer.position_embeddings.weight torch.Size([512, 1024])
transformer.lang_embeddings.weight torch.Size([2, 1024])
transformer.embeddings.weight torch.Size([64139, 1024])
transformer.layer_norm_emb.weight torch.Size([1024])
transformer.layer_norm_emb.bias torch.Size([1024])
transformer.attentions.0.q_lin.weight torch.Size([1024, 1024])
transformer.attentions.0.q_lin.bias torch.Size([1024])
transformer.attentions.0.k_lin.weight torch.Size([1024, 1024])
transformer.attentions.0.k_lin.bias torch.Size([1024])
transformer.attentions.0.v_lin.weight torch.Size([1024, 1024])
transformer.attentions.0.v_lin.bias torch.Size([1024])
transformer.attentions.0.out_lin.weight torch.Size([1024, 1024])
transformer.attentions.0.out_lin.bias torch.Size([1024])
transformer.attentions.1.q_lin.weight torch.Size([1024, 1024])
transformer.attentions.1.q_lin.bias torch.Size([1024])
transformer.attentions.1.k_lin.weight torch.Size([1024, 1024])
transformer.atten

# Train the model

In [19]:
train_model(model, data_small, epochs, False)


Training transformer
epoch 1


In [61]:
model.state_dict()

OrderedDict([('e.embedder.embed.weight',
              tensor([[ 0.0293, -0.0505, -0.1264,  ...,  0.0886,  0.0744,  0.0681],
                      [-0.1251,  0.0074,  0.0449,  ..., -0.1479, -0.0414,  0.0167],
                      [-0.0037, -0.0045,  0.0014,  ..., -0.0029, -0.0045,  0.0022],
                      ...,
                      [-0.0028, -0.0058,  0.0037,  ..., -0.0002,  0.0015,  0.0047],
                      [ 0.0061,  0.0048, -0.0063,  ..., -0.0049,  0.0059, -0.0027],
                      [ 0.0066,  0.0041, -0.0011,  ..., -0.0007, -0.0013,  0.0041]])),
             ('e.pe.pe',
              tensor([[[ 0.0000e+00,  1.0000e+00,  0.0000e+00,  ...,  1.0000e+00,
                         0.0000e+00,  1.0000e+00],
                       [ 8.4147e-01,  5.4030e-01,  8.0196e-01,  ...,  1.0000e+00,
                         1.0746e-08,  1.0000e+00],
                       [ 9.0930e-01, -4.1615e-01,  9.5814e-01,  ...,  1.0000e+00,
                         2.1492e-08,  1.0000e+00],
 

In [ ]:
filename = 'weights\EN-FR_translation_weights.pt'
torch.save(model.state_dict(), filename)

# Get model weights manually

In [56]:
model_weights = {}
for name, param in model.named_parameters():
    model_weights[name] = param.detach().numpy()
    



In [60]:
model_weights

{'e.embedder.embed.weight': array([[ 0.02931827, -0.05047134, -0.12636882, ...,  0.08857398,
          0.07438569,  0.0681239 ],
        [-0.12514947,  0.00738456,  0.04487332, ..., -0.14785293,
         -0.04139828,  0.01667465],
        [-0.00369383, -0.00450614,  0.00142797, ..., -0.00294809,
         -0.0045255 ,  0.00218448],
        ...,
        [-0.00276193, -0.00577555,  0.00368445, ..., -0.00019603,
          0.00145789,  0.00467178],
        [ 0.00605307,  0.00480419, -0.00631951, ..., -0.00494351,
          0.00591175, -0.00265854],
        [ 0.00659149,  0.0041262 , -0.00106301, ..., -0.00070657,
         -0.00125601,  0.00410483]], dtype=float32),
 'e.e_layers.0.MHA.linear.weight': array([[-0.00163168,  0.1163315 ,  0.01785881, ..., -0.03793719,
          0.04372055, -0.11165092],
        [-0.10514726,  0.05250676,  0.14395094, ..., -0.00104818,
          0.00050019, -0.03579386],
        [ 0.03854384, -0.0133293 , -0.07152691, ...,  0.08485288,
         -0.1299423 ,  0.11

ValueError: Per-column arrays must each be 1-dimensional

In [34]:
path = 'C:\\Users\micha\OneDrive\Skrivebord\MT_HW2\weights\EN-FR_weights.csv'

columns = list(df_weights.keys())

df_weights.to_csv(path, columns=columns)